In [32]:
from aoc import *
from copy import deepcopy
from collections import defaultdict, Counter
import re
from z3 import Ints, Solver, sat

year = 2024
day = 17

download_input(year, day)


In [36]:
aoc, lines, G, R, C = read_input(day, test=False)

['Program: 2,4,1,5,7,5,0,3,4,1,1,6,5,5,3,0']
3 20


In [38]:
registers = defaultdict(int)

opcodes = {0:'adv',1:'bxl',2:'bst',3:'jnz',4:'bxc',5:'out',6:'bdv',7:'cdv'}

steps = [int(x) for x in lines[0].split( )[1].split(",")]
print(steps)

instructions = [(steps[2*i],steps[2*i+1]) for i in range(len(steps)//2)]

pattern = r"Register (.*): (\d+)"
for line in aoc.split("\n")[:3]:
    matches = re.match(pattern, line)
    k, v = matches.groups()
    registers[k] = int(v)

print(registers)

[2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6, 5, 5, 3, 0]
defaultdict(<class 'int'>, {'A': 47719761, 'B': 0, 'C': 0})


In [46]:
import math
for line in aoc.split("\n")[:3]:
    matches = re.match(pattern, line)
    k, v = matches.groups()
    registers[k] = int(v)

# Combo operands 0 through 3 represent literal values 0 through 3.
# Combo operand 4 represents the value of register A.
# Combo operand 5 represents the value of register B.
# Combo operand 6 represents the value of register C.
# Combo operand 7 is reserved and will not appear in valid programs.

def determine_combo_operand(operand):
    if operand <= 3:
        return operand
    elif operand == 4:
        return registers['A']
    elif operand == 5:
        return registers['B']
    elif operand == 6:
        return registers['C']
    else:
        assert False

output = []
i = 0
while i < len(steps):
    opcode = steps[i]
    operand = steps[i+1]
    combo_operand = determine_combo_operand(operand)

    # combo
    if opcode == 0:
        registers['A'] = int(registers['A'] // math.pow(2,combo_operand))
    # literal
    if opcode == 1:
        registers['B'] = registers['B'] ^ operand
    # combo
    if opcode == 2:
        registers['B'] = combo_operand % 8
    # literal
    if opcode == 3:
        if registers['A'] == 0:
            pass
        else:
            i = operand
            continue
    # -
    if opcode == 4:
        registers['B'] = registers['B'] ^ registers['C']
    # combo
    if opcode == 5:
        output.append(combo_operand % 8)
    # combo
    if opcode == 6:
        registers['B'] = int(registers['A'] // math.pow(2,combo_operand))
    # combo
    if opcode == 7:
        registers['C'] = int(registers['A'] // math.pow(2,combo_operand))

    i+=2

#print('finished')
#print(output)
#print(','.join([np.base_repr(x,base=3) for x in output]))
print(','.join(str(x) for x in output))


7,0,3,1,2,6,3,7,1


In [448]:

def test_register_value(A_new):

    registers['A'] = A_new

    output = []
    i = 0
    while i < len(steps):
        opcode = steps[i]
        operand = steps[i+1]
        combo_operand = determine_combo_operand(operand)

        # combo
        if opcode == 0:
            registers['A'] = int(registers['A'] // math.pow(2,combo_operand))
        # literal
        if opcode == 1:
            registers['B'] = registers['B'] ^ operand
        # combo
        if opcode == 2:
            registers['B'] = combo_operand % 8
        # literal
        if opcode == 3:
            if registers['A'] == 0:
                pass
            else:
                i = operand
                continue
        # -
        if opcode == 4:
            registers['B'] = registers['B'] ^ registers['C']
        # combo
        if opcode == 5:
            output.append(combo_operand % 8)
        # combo
        if opcode == 6:
            registers['B'] = int(registers['A'] // math.pow(2,combo_operand))
        # combo
        if opcode == 7:
            registers['C'] = int(registers['A'] // math.pow(2,combo_operand))

        i+=2

    return output

target = [x for x in steps]

My starting program is this.

Program: 2,4,1,5,7,5,0,3,4,1,1,6,5,5,3,0

2 4 - set B to A % 8
1 5 - set B to B XOR 5
7 5 - set C to A // (2 ^ B)
0 3 - set A to A // (2 ^ A)
4 1 - set B to B XOR C
1 6 - set B to B XOR 6
5 5 - output B % 8
3 0 - if A > 0 jump to start.


def step(A,B,C):
    B = A % 8
    B = B ^ 5
    C =

A = A, C = 0
1. B = A % 8
2. B = (A % 8) XOR 5,
3. C = A // (2 ^ B)
4. B = (A % 8) XOR 5 XOR C
5. B = B XOR 6
5. output.

So B is output as A % 8 then XOR 5 then XOR A // (2 ^ B) then XOR 6.

3 bit number XOR 5
A A XOR 5
0 5
1 4
2 7
3 6
4 1
5 0
6 3
7



In [450]:
def step(A_in,B_in,C_in, debug = False):
    # 2 4 - set B to A % 8
    B_in = A_in % 8
    if debug:
        print(f"{A_in=} {B_in=} {C_in=}")

    # 1 5 - set B to B XOR 5
    B_in = B_in ^ 5
    if debug:
        print(f"{A_in=} {B_in=} {C_in=}")

    # 7 5 - set C to A // (2 ^ B)
    C_in = A_in // int(math.pow(2,B_in))
    if debug:
            print(f"{A_in=} {B_in=} {C_in=}")

    # 0 3 - set A to A // (2 ^ 3)
    # 2 ^ 3 == 8
    A_in = A_in // 8
    if debug:
        print(f"{A_in=} {B_in=} {C_in=}")

    # 4 1 - set B to B XOR C
    B_in = B_in ^ C_in
    if debug:
        print(f"{A_in=} {B_in=} {C_in=}")

    # 1 6 - set B to B XOR 6
    B_in = B_in ^ 6
    if debug:
        print(f"{A_in=} {B_in=} {C_in=}")

    # 5 5 - output B % 8
    #print("OUTPUT", B_in % 8)
    return A_in, B_in, C_in


A = 47719761
B = 0
C = 0
out = []

while True:
    A, B, C = step(A,B,C)
    out.append(B % 8)
    if A == 0:
        break

print(out)

[7, 0, 3, 1, 2, 6, 3, 7, 1]


defaultdict(<class 'list'>, {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: []})
[2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6, 5, 5, 3, 0]


[6, 5, 5, 3, 0]

In [476]:

def test_next(inp='000', len_tar=1):
    """
    Test things 3 at a time.
    :param inp: default input to start with.
    :param len_tar: partial result is ok if it matches the target up to this length.
    :return: all results which work up to this length.
    """

    results = []
    outputs = []

    test_a = inp

    for i in range(0,8):

        test_a += format(i,"03b")
        test_a = int(test_a, 2)

        A = test_a
        B = 0
        C = 0
        out = []

        while True:
            A, B, C = step(A,B,C)
            out.append(B % 8)
            if A == 0:
                break

        if out == target[-len_tar:]:
            results.append(np.base_repr(test_a, base=2))
            outputs.append(out)

        test_a = inp

    return results

found = defaultdict(list)

found[0] = test_next()

j = 0

finished = False
while not finished: #len(found) < len(target):
    for result in found[j]:
        next_results = test_next(result, j+2)
        found[j+1].extend(next_results)

        for x in next_results:
            if test_register_value(int(x,2)) == target:
                print(int(x,2))
                finished = True
                break
        if finished:
            break

    j += 1


109020013201563
